# Định giá P/B và P/E

In [ ]:
# Bước 1: Nạp dữ liệu
import pandas as pd
from vnstock import Finance
symbol = "VPB"
period = 'quarter'
finance = Finance(symbol=symbol, source='VCI')
df = finance.ratio(period=period, lang='vi')
df.columns = df.columns.droplevel()
df = df[['CP', 'Năm', 'Kỳ', 'EPS (VND)', 'P/E', 'BVPS (VND)','P/B',]]

# Lọc dữ liệu cho năm hiện tại và năm liền trước
current_year = pd.Timestamp.now().year
df = df[df['Năm'].isin([current_year, current_year - 1])]
#df


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


In [14]:
# Bước 2: Tính P/E và P/B trung bình lịch sử của cổ phiếu
pe_mean = df['P/E'].mean()
pb_mean = df['P/B'].mean()
pe_median = df['P/E'].median()
pb_median = df['P/B'].median()

# print(f"P/E trung bình: {pe_mean:.2f}")
# print(f"P/B trung bình: {pb_mean:.2f}")

In [15]:
# Bước 3: Tinh P/E và P/B trung bình của ngành

# Xác định ngành của cổ phiếu
from vnstock import Company, Listing
company = Company(symbol=symbol, source='VCI')
industry = company.overview()['icb_name4']
industry = industry[0]
#industry

# Lấy danh sách các cổ phiếu trên sàn HOSE trong cùng ngành
listing = Listing(source='vci')
df_industry = listing.symbols_by_industries()
df_exchange = listing.symbols_by_exchange()

hose_syms = df_exchange.query('exchange == "HSX" & type == "STOCK"')['symbol']
industry_syms = df_industry[df_industry['icb_name4'] == industry]['symbol']
symbols = industry_syms[industry_syms.isin(hose_syms)].tolist()
# Lấy dữ liệu P/E và P/B gần nhất của các cổ phiếu trong ngành
pe_pb_data = []
for sym in symbols:
    try:
        finance = Finance(symbol=sym, source='VCI')
        df_temp = finance.ratio(period=period, lang='vi')
        df_temp.columns = df_temp.columns.droplevel()
        pe_latest = df_temp['P/E'].iloc[0]
        pb_latest = df_temp['P/B'].iloc[0]
        pe_pb_data.append({'symbol': sym, 'PE_latest': pe_latest, 'PB_latest': pb_latest})
    except:
        continue

pe_pb_df = pd.DataFrame(pe_pb_data)


DataFrame.applymap has been deprecated. Use DataFrame.map instead.
Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behav

In [16]:
# Tính P/E và P/B trung bình của ngành
industry_pe_mean = pe_pb_df['PE_latest'].mean()
industry_pb_mean = pe_pb_df['PB_latest'].mean()
industry_pe_median = pe_pb_df['PE_latest'].median()
industry_pb_median = pe_pb_df['PB_latest'].median()
# print(f"P/E trung bình ngành {industry}: {industry_pe_mean:.2f}")
# print(f"P/B trung bình ngành {industry}: {industry_pb_mean:.2f}")

In [17]:
# Bước 4: Định giá cổ phiếu

# Định giá cổ phiếu dựa trên P/E và P/B lịch sử
eps_latest = df['EPS (VND)'].iloc[0]
bvps_latest = df['BVPS (VND)'].iloc[0]


# Tính giá trị hợp lý của cổ phiếu dựa trên P/E và P/B
fair_price_pe = eps_latest * pe_mean
fair_price_pb = bvps_latest * pb_mean
fair_price_pe_median = eps_latest * pe_median
fair_price_pb_median = bvps_latest * pb_median
# print(f"Giá trị hợp lý của cổ phiếu {symbol} dựa trên P/E lịch sử: {fair_price_pe:.2f} VND")
# print(f"Giá trị hợp lý của cổ phiếu {symbol} dựa trên P/B lịch sử: {fair_price_pb:.2f} VND")

# Định giá cổ phiếu SSI dựa trên P/E và P/B ngành
fair_price_pe_industry = eps_latest * industry_pe_mean
fair_price_pb_industry = bvps_latest * industry_pb_mean
fair_price_pe_median_industry = eps_latest * industry_pe_median
fair_price_pb_median_industry = bvps_latest * industry_pb_median
# print(f"Giá trị hợp lý của cổ phiếu {symbol} dựa trên P/E ngành: {fair_price_pe_industry:.2f} VND")
# print(f"Giá trị hợp lý của cổ phiếu {symbol} dựa trên P/B ngành: {fair_price_pb_industry:.2f} VND")

In [18]:
# Tính P/E và P/B hiện tại
from datetime import datetime
today = datetime.now().strftime('%Y-%m-%d')
yesterday = (datetime.now() - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
from vnstock import Quote
quote = Quote(symbol=symbol, source='VCI')
df_price = quote.history(start=yesterday, end=today)
price = df_price['close'].iloc[-1]
price = float(price) * 1000
#price


In [19]:
valuation_pe_df = pd.DataFrame({
    'CP': [symbol],
    'EPS gần nhất': [df['EPS (VND)'].iloc[0]],
    'P/E gần nhất': price / df['EPS (VND)'].iloc[0],
    'P/E lịch sử': [pe_mean],
    'Giá hợp lý theo P/E lịch sử': fair_price_pe,
    'P/E trung vị': [pe_median],
    'Giá hợp lý theo P/E trung vị': fair_price_pe_median,
    'P/E ngành': [industry_pe_mean],
    'Giá hợp lý theo P/E ngành': fair_price_pe_industry,
    'P/E trung vị ngành': [industry_pe_median],
    'Giá hợp lý theo P/E trung vị ngành': fair_price_pe_median_industry,
})
valuation_pb_df = pd.DataFrame({
    'CP': [symbol],
    'BVPS gần nhất': [df['BVPS (VND)'].iloc[0]],
    'P/B gần nhất': price / df['BVPS (VND)'].iloc[0],
    'P/B lịch sử': [pb_mean],
    'Giá hợp lý theo P/B lịch sử': fair_price_pb,
    'P/B trung vị': [pb_median],
    'Giá hợp lý theo P/B trung vị': fair_price_pb_median,
    'P/B ngành': [industry_pb_mean],
    'Giá hợp lý theo P/B ngành': fair_price_pb_industry,
    'P/B trung vị ngành': [industry_pb_median],
    'Giá hợp lý theo P/B trung vị ngành': fair_price_pb_median_industry,
})

In [20]:
fair_value_pe_df = pd.DataFrame({
    'CP': [symbol],
    'Giá hợp lý theo P/E lịch sử': [df['EPS (VND)'].iloc[0] * pe_mean],
    'Giá hợp lý theo P/E ngành': [df['EPS (VND)'].iloc[0] * industry_pe_mean],
    'Giá hợp lý theo P/E trung vị': [df['EPS (VND)'].iloc[0] * pe_median],
    'Giá hợp lý theo P/E trung vị ngành': [df['EPS (VND)'].iloc[0] * industry_pe_median],
})
fair_value_pb_df = pd.DataFrame({
    'CP': [symbol],
    'Giá hợp lý theo P/B lịch sử': [df['BVPS (VND)'].iloc[0] * pb_mean],
    'Giá hợp lý theo P/B ngành': [df['BVPS (VND)'].iloc[0] * industry_pb_mean],
    'Giá hợp lý theo P/B trung vị': [df['BVPS (VND)'].iloc[0] * pb_median],
    'Giá hợp lý theo P/B trung vị ngành': [df['BVPS (VND)'].iloc[0] * industry_pb_median],
})

In [ ]:
valuation_pe_df

In [ ]:
fair_value_pe_df

In [21]:
valuation_pb_df

,CP,BVPS gần nhất,P/B gần nhất,P/B lịch sử,Giá hợp lý theo P/B lịch sử,P/B trung vị,Giá hợp lý theo P/B trung vị,P/B ngành,Giá hợp lý theo P/B ngành,P/B trung vị ngành,Giá hợp lý theo P/B trung vị ngành
0,VPB,19406.685361,1.445378,1.236742,24001.067127,1.142726,22176.522416,1.413206,27425.645999,1.377527,26733.228931


In [22]:
fair_value_pb_df

,CP,Giá hợp lý theo P/B lịch sử,Giá hợp lý theo P/B ngành,Giá hợp lý theo P/B trung vị,Giá hợp lý theo P/B trung vị ngành
0,VPB,24001.067127,27425.645999,22176.522416,26733.228931
